datasrc: https://drive.google.com/open?id=16V_ZlkW4SsnNDtnGmaBRq2OoPmUOc5mY

more datasrc:
https://github.com/renmengye/few-shot-ssl-public#miniimagenet

#Setup

In [ ]:
!pip install wandb

In [ ]:
%cd /content
!git clone https://github.com/ethan-jiang-1/pim_classification.git pimcls 
%cd /content/pimcls
!git checkout master
!git pull origin master
!git submodule update --init --recursive
%cd /content/pimcls/pim
!pip install -e .
%cd /content/pimcls

In [ ]:
#!pip install google-api-python-client -t app/lib
#!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install googledrivedownloader

#Download dataset - imagenette2

In [ ]:
import cv2
import os
import pickle
from tqdm import tqdm

def dump_pickle_to_files(name):
    pkl_name = "mini-imagenet-cache-{}.pkl".format(name)
    subfolder = "miniimagenet/{}".format(name)
    #if os.path.isdir(subfolder):
    #    return
    
    os.makedirs(subfolder, exist_ok=True)
    with open(pkl_name, "rb") as f:
      pobj = pickle.load(f)

      print(len(pobj["image_data"]))
      print(len(pobj["class_dict"]))

      #key_list = list(pobj["class_dict"].keys())
      for key, vals in tqdm(pobj["class_dict"].items(), desc=name):
          imgfolder = "{}/{}".format(subfolder, key)
          if not os.path.isdir(imgfolder):
              os.makedirs(imgfolder, exist_ok=True)
          for val in vals:
              img = pobj["image_data"][val]
              #print(img.shape)
              filename = "{}/{}.png".format(imgfolder, val)
              cv2.imwrite(filename, img)


def dump_all_pickle_to_files():
    for name in ["train", "test", "val"]:
        dump_pickle_to_files(name)

In [ ]:
import os
import time
from IPython.display import clear_output
from google_drive_downloader import GoogleDriveDownloader as gdd

%cd /content/pimcls

google_download = False
if google_download:
    # https://drive.google.com/open?id=16V_ZlkW4SsnNDtnGmaBRq2OoPmUOc5mY
    if not os.path.isfile("miniimagenet.tgz"):
        gdd.download_file_from_google_drive(file_id='16V_ZlkW4SsnNDtnGmaBRq2OoPmUOc5mY',
                                            dest_path='/content/pimcls/miniimagenet.tgz')

    if not os.path.isdir("miniimagenet"):
        !tar -xvzf miniimagenet.tgz
        time.sleep(10)
        clear_output(wait=True)
else:
    if not os.path.isfile("miniimagenet.zip"):
        !wget https://data.deepai.org/miniimagenet.zip
    if not os.path.isfile("mini-imagenet-cache-train.pkl"):
        !unzip miniimagenet.zip
    dump_all_pickle_to_files()

In [ ]:
import pickle

def inspect_pickle_object(name, pobj):
    print()  
    print(name, type(pobj))
    for key in pobj.keys():
        print(key)
    print(len(pobj["image_data"]))
    print(len(pobj["class_dict"]))

    print(pobj["image_data"].shape)

    key_list = list(pobj["class_dict"].keys())
    print(len(key_list), key_list)
    keyname = key_list[0]
    print(pobj["class_dict"][keyname])

    #x_pobj = pobj["image_data"].reshape([20, 600, 84, 84, 3])
    #print(x_pobj.shape)


with open("mini-imagenet-cache-test.pkl", "rb") as test_in:
    ptest = pickle.load(test_in)
    inspect_pickle_object("test", ptest)
with open("mini-imagenet-cache-train.pkl", "rb") as train_in:
    ptrain = pickle.load(train_in)
    inspect_pickle_object("train", ptrain)
with open("mini-imagenet-cache-val.pkl", "rb") as val_in:
    pval = pickle.load(val_in)
    inspect_pickle_object("val", pval)

In [ ]:
pause

#Prepare training

In [ ]:
cmd = """./train.py ../miniimagenet 
--model seresnet34 
--sched cosine 
--epochs 150 
--warmup-epochs 5 
--lr 0.4 
--reprob 0.5 
--remode pixel 
--batch-size 256 
--amp 
--log-wandb 
-j 4"""

cmd_line = cmd.replace("\n", "")
print(cmd_line)

#Training

In [ ]:
%cd /content/pimcls/pim 
!$cmd_line